In [1]:

import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../maxtext')))
os.environ["SKIP_JAX_PRECOMPILE"] = "1"

import jax.numpy as jnp
from flax import nnx
import sys
import os
import flax.linen as nn
import logging

import MaxText as mt
from MaxText import pyconfig
from tunix.rl.rollout.vllm_rollout import VllmRollout
from tunix.rl.rollout import base_rollout
import transformers
import jax 
from MaxText.integration.tunix.tunix_adaptor import TunixMaxTextLlama
from tunix.rl import utils


/home/wenxindong_google_com/miniconda3/envs/tunix/lib/python3.12/site-packages/jax/_src/cloud_tpu_init.py:84: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
2025-08-12 18:25:50.679688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755023150.692539  202297 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755023150.696268  202297 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:0

INFO 08-12 18:25:55 [__init__.py:241] Automatically detected platform tpu.
WARNING 08-12 18:25:55 [__init__.py:14] 🚨  CAUTION: You are using 'tpu_commons' , which is experimental and NOT intended for production use yet. Please see the README for more details.
INFO 08-12 18:25:55 [__init__.py:16] TPU info: node_name=wenxindong-v6e-8 | tpu_type=v6e-8 | worker_id=0 | num_chips=8 | num_cores_per_chip=1
INFO 08-12 18:25:55 [__init__.py:29] Running vLLM without Pathways. Module pathwaysutils is not imported.


In [2]:

show_hbm_usage = utils.show_hbm_usage

show_hbm_usage("Before loading model")
def get_ref_maxtext_model():

  #TODO: @mazumdera: change this to use Gemma2-2b-it
  config = pyconfig.initialize(
      ["", "../../maxtext/MaxText/configs/base.yml"], #TODO: @mazumdera: why decode.py?
      base_output_directory="gs://dummy_output_dir",  # This is not used in Tunix.
      run_name="none",
      tokenizer_path="../../maxtext/assets/tokenizer.gemma",
      per_device_batch_size=1,
      max_target_length=1024,
      steps=10,
      async_checkpointing="false",
      model_name="llama3.1-8b", #"llama3.1-8b"
      checkpoint_period=5, 
      skip_jax_distributed_system="true",
      weight_dtype="bfloat16",
      attention="dot_product"
  )
  
  def create_model(config):
    return mt.from_pretrained(config, rngs=nnx.Rngs(params=0, dropout=1))

  model = nnx.eval_shape(create_model, config=config)

  abstract_model = nnx.eval_shape(create_model, config=config)
  graphdef, abstract_state = nnx.split(abstract_model)
  print('The abstract NNX state (all leaves are abstract arrays):')
  nnx.display(abstract_state)

  @nnx.jit
  def partial_init(config):
    model = create_model(config)
    # nnx.update(model, checkpoint)
    # shard model
    state = nnx.state(model)
    specs = nnx.get_partition_spec(state)
    state = jax.lax.with_sharding_constraint(state, specs)
    nnx.update(model, state)
    return model

  with jax.sharding.use_mesh(model.mesh), nn.logical_axis_rules(config.logical_axis_rules):
    model = partial_init(config)
  print(model)

  tunix_model = TunixMaxTextLlama(
        base_model=model,
        use_attention_mask=False,  # trust Tunix loss masking
    )
  mesh  = tunix_model.base.mesh
  
  tunix_model.to_hf_mappings = lambda *args: {}
  tunix_model.to_hf_transpose_keys = lambda *args: {}
  tunix_model.lora_to_hf_mappings = lambda *args: {}

  # Add these lines to properly get the graph definition and state
  graphdef, state = nnx.split(tunix_model)
  tunix_model = nnx.merge(graphdef, state)  # Recreate model in proper NNX format
    
  return tunix_model, mesh

model, mesh = get_ref_maxtext_model()

print(model)
show_hbm_usage("After loading model")


TOTAL_GENERATION_STEPS = 64
MAX_PROMPT_LENGTH = 64  
TEMPERATURE = 0.9
TOP_P = 1.0
TOP_K = None
cache_config = base_rollout.RolloutConfig(max_tokens_to_generate=TOTAL_GENERATION_STEPS, max_prompt_length=MAX_PROMPT_LENGTH, kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256, temperature=TEMPERATURE, top_p=TOP_P, top_k=TOP_K)




Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_0(process=0,(0,0,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_1(process=0,(1,0,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_2(process=0,(0,1,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_3(process=0,(1,1,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_4(process=0,(0,2,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_5(process=0,(1,2,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_6(process=0,(0,3,0,0))
Using %s / %s (%s) on %s 32.1 KiB 31.2 GiB 9.804998794404818e-07 TPU_7(process=0,(1,3,0,0))
Updating keys from env and command line: ['run_name', 'model_name', 'async_checkpointing', 'checkpoint_period', 'weight_dtype', 'attention', 'base_output_directory', 'tokenizer_path', 'per_device_batch_size', 'steps', 'skip_jax_distributed_system', 'max_target_len

Num_devices: 8, shape (1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Transformer( # Param: 8,030,261,248 (16.1 GB), RngState: 4 (24 B), Total: 8,030,261,252 (16.1 GB)
  config=<MaxText.pyconfig.HyperParameters object at 0x7c05b37dbfe0>,
  decoder=ToNNX( # Param: 7,504,924,672 (15.0 GB), RngState: 4 (24 B), Total: 7,504,924,676 (15.0 GB)
    decoder_norm={'scale': Param( # 4,096 (8.2 KB)
      value=Array(shape=(4096,), dtype=dtype(bfloat16)),
      mesh=None,
      sharding=('norm',),
      sharding_rules=None,
      linen_meta_type=LogicallyPartitioned
    )},
    layers={'mlp': {'wi_0': {'kernel': Param( # 1,879,048,192 (3.8 GB)
      value=Array(shape=(4096, 32, 14336), dtype=dtype(bfloat16)),
      mesh=None,
      sharding=('embed', 'layers', 'mlp'),
      sharding_rules=None,
      linen_meta_type=LogicallyPartitioned
    )}, 'wi_1': {'kernel': Param( # 1,879,048,192 (3.8 GB)
      value=Array(shape=(4096, 32, 14336), dtype=dtype(bfloat16)),
      mesh=None,
      sharding=('embed', 'layer

In [3]:
nnx.display(nnx.state(model))

In [4]:
def create_maxtext_to_vllm_mappings():
    """Create mappings for transferring MaxText scanned state to vLLM unscanned state."""
    return {
        # Token embeddings - shard vocab dimension for TP
        'base.token_embedder.embedding': ('embed.embedding', ('model', None)),
        
        # Final layer norm - no sharding needed
        'base.decoder.decoder_norm.scale': ('model.norm.scale', (None,)),
        
        # LM head (logits projection) - shard vocab dimension for TP
        'base.decoder.logits_dense.kernel': ('lm_head', (None, 'model')),
        
        # Layer-specific mappings (scanned -> unscanned)
        # MLP components - shard hidden dimensions for TP
        'base.decoder.layers.mlp.wi_0.kernel': ('model.layers.*.mlp.gate_proj.kernel', (None, 'layer', 'model')),  # gate_proj: (4096, 14336) - shard output
        'base.decoder.layers.mlp.wi_1.kernel': ('model.layers.*.mlp.up_proj.kernel', (None, 'layer', 'model')),    # up_proj: (4096, 14336) - shard output  
        'base.decoder.layers.mlp.wo.kernel': ('model.layers.*.mlp.down_proj.kernel', ('model', 'layer', None)),    # down_proj: (14336, 4096) - shard input
        
        # Layer norms - no sharding needed
        'base.decoder.layers.pre_self_attention_layer_norm.scale': ('model.layers.*.input_layernorm.scale', (None, 'layer')),
        'base.decoder.layers.post_self_attention_layer_norm.scale': ('model.layers.*.post_attention_layernorm.scale', (None, 'layer')),
        
        # Attention components - shard head dimensions for TP
        'base.decoder.layers.self_attention.query.kernel': ('model.layers.*.self_attn.q_proj.kernel', (None, 'layer', 'model', None)),  # q_proj: shard num_heads
        'base.decoder.layers.self_attention.key.kernel': ('model.layers.*.self_attn.k_proj.kernel', (None, 'layer', 'model', None)),    # k_proj: shard num_kv_heads
        'base.decoder.layers.self_attention.value.kernel': ('model.layers.*.self_attn.v_proj.kernel', (None, 'layer', 'model', None)),  # v_proj: shard num_kv_heads
        'base.decoder.layers.self_attention.out.kernel': ('model.layers.*.self_attn.o_proj.kernel', ('model', 'layer', None, None)),    # o_proj: shard input heads
    }



In [5]:
from functools import lru_cache


In [7]:
# # Debug

# from tunix.generate import vllm_sampler

# sampler = vllm_sampler.VllmSampler(
#     tokenizer=model_tokenizer,
#     max_model_len=64,
#     mesh=mesh,
#     model_version="meta-llama/Llama-3.1-8B",
#     hbm_utilization=0.3,
#     mapping_config=vllm_sampler.MappingConfig(
#         to_hf_mappings=mappings,
#         to_hf_transpose_keys={},
#         lora_to_hf_mappings=None,
#         lora_config=None,
#     ),
#     tp = 8,
# )


# # Define transpose operations needed for shape compatibility
# transpose_keys = {
#     # # MLP transposes (after layer extraction)
#     # 'wo.kernel': (1, 0),  # down_proj: (14336, 4096) - transpose needed
    
#     # # Attention output transpose (after layer extraction) 
#     # 'out.kernel': (1, 2, 0),  # o_proj: (32, 128, 4096) -> (32, 128, 4096) - reorder dimensions
# }

# from tunix.generate.utils import transfer_state_with_mappings_scanned
# transfer_state_with_mappings(
#     nnx.state(model),
#     sampler._model_runner.state,
#     mappings,
#     transpose_keys=transpose_keys,
# )


In [8]:
mappings = create_maxtext_to_vllm_mappings()

transpose_keys = {
    # # MLP transposes (after layer extraction)
    # 'wo.kernel': (1, 0),  # down_proj: (14336, 4096) - transpose needed
    
    # # Attention output transpose (after layer extraction) 
    # 'out.kernel': (1, 2, 0),  # o_proj: (32, 128, 4096) -> (32, 128, 4096) - reorder dimensions
}


model.to_hf_mappings = create_maxtext_to_vllm_mappings
model.to_hf_transpose_keys = lambda *args: transpose_keys
model.lora_to_hf_mappings = lambda *args: None  # No LoRA

In [9]:
model_tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")

In [10]:
rollout = VllmRollout(model=model,tokenizer=model_tokenizer,cache_config_or_size=64, mesh=mesh,lora_config=None,model_version="meta-llama/Llama-3.1-8B")


INFO 08-12 18:26:35 [utils.py:326] non-default args: {'model': 'meta-llama/Llama-3.1-8B', 'max_model_len': 64, 'gpu_memory_utilization': 0.3, 'disable_log_stats': True}
WARNING 08-12 18:26:35 [__init__.py:516] The global random seed is set to 0. Since VLLM_ENABLE_V1_MULTIPROCESSING is set to False, this may affect the random state of the Python process that launched vLLM.
INFO 08-12 18:26:45 [__init__.py:707] Resolved architecture: LlamaForCausalLM
INFO 08-12 18:26:45 [__init__.py:1731] Using max model len 64


2025-08-12 18:26:45,234	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 08-12 18:26:45 [__init__.py:2031] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 08-12 18:26:45 [tpu_jax.py:140] The model dtype is not properly set for JAX backend. Overwriting it to bfloat16


/home/wenxindong_google_com/miniconda3/envs/tunix/lib/python3.12/site-packages/torch_xla/__init__.py:258: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


INFO 08-12 18:26:46 [tpu_jax.py:177] Force using UniProcExecutor for JAX on single host.
INFO 08-12 18:26:47 [core.py:72] Initializing a V1 LLM engine (v0.1.dev8444+g094bed7da.d20250811) with config: model='meta-llama/Llama-3.1-8B', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=<class 'jax.numpy.bfloat16'>, max_seq_len=64, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=None, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_

In [13]:
from tunix.rl.rollout.base_rollout import RolloutConfig



rollout.generate(["hello world", "how are you?"], rollout_config= RolloutConfig(n=1))

WARNING 08-12 18:29:16 [__init__.py:1606] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Processed prompts: 100%|██████████| 2/2 [00:03<00:00,  1.98s/it, est. speed input: 2.53 toks/s, output: 29.87 toks/s]


RolloutOutput(text=['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'], logits=None, tokens=Array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 128001, 128001, 128001, 128001],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

hi
